In [15]:
import openai

client = openai.OpenAI()
messages = []

In [16]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print(f"AI: {message}")

In [17]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 
AI: Hello! How can I assist you today?
User: 
AI: It seems that your message is empty. How can I help you today?
